## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-23-10-55-23 +0000,nypost,Former Des Moines schools chief pleads guilty ...,https://nypost.com/2026/01/23/us-news/former-d...
1,2026-01-23-10-54-01 +0000,bbc,Andy Burnham faces decision on bid to return a...,https://www.bbc.com/news/articles/clyeeqyvv78o...
2,2026-01-23-10-49-34 +0000,nyt,"Ukrainian, Russian and U.S. Officials to Meet ...",https://www.nytimes.com/2026/01/23/world/europ...
3,2026-01-23-10-48-45 +0000,nyt,Trump Says Canada Is No Longer Welcome on Peac...,https://www.nytimes.com/live/2026/01/22/us/tru...
4,2026-01-23-10-46-56 +0000,nyt,"After Trump’s Ultimatum, Greenland Talks Inclu...",https://www.nytimes.com/2026/01/22/world/europ...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2439/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
26,trump,80
153,new,23
33,greenland,22
187,ice,21
119,up,17


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
366,2026-01-22-11-19-40 +0000,nypost,"Trump claims ‘Iran does want to talk, and we’l...",https://nypost.com/2026/01/22/us-news/trump-cl...,158
335,2026-01-22-14-32-36 +0000,nypost,Trump vows US will have ‘total access’ to Gree...,https://nypost.com/2026/01/22/us-news/trump-vo...,153
56,2026-01-23-07-44-18 +0000,nypost,Trump’s border czar Tom Homan tears new Dem Vi...,https://nypost.com/2026/01/23/us-news/trumps-b...,150
233,2026-01-22-21-19-43 +0000,bbc,"Board of Peace, Zelensky and Musk - What happe...",https://www.bbc.com/news/videos/c78vvvn4qnlo?a...,149
225,2026-01-22-21-40-18 +0000,bbc,Could Trump's new Board of Peace sideline the ...,https://www.bbc.com/news/articles/cn8jj228g2vo...,141


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
366,158,2026-01-22-11-19-40 +0000,nypost,"Trump claims ‘Iran does want to talk, and we’l...",https://nypost.com/2026/01/22/us-news/trump-cl...
335,73,2026-01-22-14-32-36 +0000,nypost,Trump vows US will have ‘total access’ to Gree...,https://nypost.com/2026/01/22/us-news/trump-vo...
56,70,2026-01-23-07-44-18 +0000,nypost,Trump’s border czar Tom Homan tears new Dem Vi...,https://nypost.com/2026/01/23/us-news/trumps-b...
256,55,2026-01-22-19-43-16 +0000,nyt,Judge Rejects DOJ’s Criminal Complaint Against...,https://www.nytimes.com/2026/01/22/us/politics...
111,54,2026-01-23-02-49-00 +0000,wsj,A resolution to rein in President Trump’s powe...,https://www.wsj.com/politics/policy/gop-lawmak...
232,52,2026-01-22-21-22-31 +0000,nyt,"Zelensky Laces Into Europe, Saying It Must Ste...",https://www.nytimes.com/2026/01/22/world/europ...
275,44,2026-01-22-18-36-55 +0000,nypost,Elon Musk’s Greenland jokes fall flat at Davos...,https://nypost.com/2026/01/22/business/elon-mu...
210,40,2026-01-22-22-02-00 +0000,wsj,JPMorgan CEO Jamie Dimon Reaps $43 Million in ...,https://www.wsj.com/finance/banking/jpmorgan-c...
121,37,2026-01-23-02-20-00 +0000,wsj,California Gov. Gavin Newsom spent his days at...,https://www.wsj.com/politics/policy/the-newsom...
49,35,2026-01-23-08-38-23 +0000,wsj,Stock Market Today: Gold Prices Hit Record; In...,https://www.wsj.com/livecoverage/stock-market-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
